In [3]:
from dotenv import load_dotenv
import pandas as pd
import os
from sqlalchemy import create_engine

load_dotenv()

database = os.getenv('CON_STR')

engine = create_engine(database)

In [9]:
sql1 = ''' 
with temp as (
	select  
		t.YYYYMM, 
		gr_cd,
		t.id,
		max(case when b.cigatype='HnB' and b.company = 'PMK' then 1 else 0 end) IQOS_Purchased,
		max(case when b.cigatype='CC' then 1 else 0 end) CC_Purchased,
		max(case when b.cigatype='HnB' and b.company != 'PMK' then 1 else 0 end) CompHnB_Purchased
	from  cu.agg_CU_TEREA_Total_Sourcing t
		join cu.Fct_BGFR_PMI_Monthly a on t.id = a.id 
			and a.YYYYMM = t.YYYYMM
		join cu.dim_product_master b on a.ITEM_CD = b.PROD_ID and CIGADEVICE =  'CIGARETTES' AND  b.cigatype != 'CSV' 
		join cu.dim_Regional_area c on t.SIDO_nm = c.sido_nm
	where 1=1
	and t.YYYYMM >= '202401'
	group BY 	    	
		t.YYYYMM, 
		gr_cd,
		t.id
	having 
	--	IQOS Only 조건만
		 max(case when b.cigatype='HnB' and b.company = 'PMK' then 1 else 0 end)  > 0 
		 and max(case when b.cigatype='CC' then 1 else 0 end) = 0 
		 and max(case when b.cigatype='HnB' and b.company != 'PMK' then 1 else 0 end) = 0
)
select t.YYYYMM, COALESCE(gr_cd, '합계')  'Gr Region',
	FLAVORSEG_type3,
	count(distinct t.id) n
from temp t
	join cu.Fct_BGFR_PMI_Monthly a on t.id = a.id 
		and a.YYYYMM = t.YYYYMM
	join cu.dim_product_master b on a.ITEM_CD = b.PROD_ID and CIGADEVICE =  'CIGARETTES' AND  b.cigatype != 'CSV' 
		and ProductSubFamilyCode = 'TEREA'
where 1=1 
group by 
	grouping sets ( 
		(t.YYYYMM, gr_cd, FLAVORSEG_type3 ),
		(t.YYYYMM, FLAVORSEG_type3 )
	)
'''

taste_df = pd.read_sql_query(sql=sql1, con=engine)

pivot_taste = taste_df.pivot_table(index=['YYYYMM', 'Gr Region'],
                     columns='FLAVORSEG_type3',
                     values='n'
                     )

pivot_taste.to_clipboard()

In [6]:
sql =''' 
-- terea sku usage
with temp as (
	select  
		t.YYYYMM, 
		gr_cd,
		t.id,
		max(case when b.cigatype='HnB' and b.company = 'PMK' then 1 else 0 end) IQOS_Purchased,
		max(case when b.cigatype='CC' then 1 else 0 end) CC_Purchased,
		max(case when b.cigatype='HnB' and b.company != 'PMK' then 1 else 0 end) CompHnB_Purchased
	from  cu.agg_CU_TEREA_Total_Sourcing t
		join cu.Fct_BGFR_PMI_Monthly a on t.id = a.id 
			and a.YYYYMM = t.YYYYMM
		join cu.dim_product_master b on a.ITEM_CD = b.PROD_ID and CIGADEVICE =  'CIGARETTES' AND  b.cigatype != 'CSV' 
		join cu.dim_Regional_area c on t.SIDO_nm = c.sido_nm
	where 1=1
	and t.YYYYMM >= '202401'
	group BY 	    	
		t.YYYYMM, 
		gr_cd,
		t.id
	having 
	--	IQOS Only 조건만
		 max(case when b.cigatype='HnB' and b.company = 'PMK' then 1 else 0 end)  > 0 
		 and max(case when b.cigatype='CC' then 1 else 0 end) = 0 
		 and max(case when b.cigatype='HnB' and b.company != 'PMK' then 1 else 0 end) = 0
)
select t.YYYYMM, COALESCE(gr_cd, '합계')  'Gr Region',
	engname,
	count(distinct t.id) n
from temp t
	join cu.Fct_BGFR_PMI_Monthly a on t.id = a.id 
		and a.YYYYMM = t.YYYYMM
	join cu.dim_product_master b on a.ITEM_CD = b.PROD_ID and CIGADEVICE =  'CIGARETTES' AND  b.cigatype != 'CSV' 
		and ProductSubFamilyCode = 'TEREA'
where 1=1 
group by 
	grouping sets ( 
		(t.YYYYMM, gr_cd, engname ),
		(t.YYYYMM, engname )
	)
;
'''

df = pd.read_sql_query(sql=sql, con=engine)

pivot_sku = df.pivot_table(index=['YYYYMM', 'Gr Region'],
                           columns= 'engname',
                           values='n'
                           )

pivot_sku.to_clipboard()